In [ ]:
from tmdbv3api import TMDb, Movie
from tmdbv3api import Account
from tmdbv3api import Authentication

##Import the rest of required libraries
import time, random
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set(palette='icefire') # set the default Seaborn style for graphics

##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47' ##insert apikey
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()
movie = Movie()

# List of functions
#### 1. `Full_Mov_DB_2_CSV(date)`
    TMDB provides daily exports of its entire database.
    However, there are not many attributes to this list.
    Takes a date param as string (YYYY-MM-DD), and writes the export for that day to a CSV file in the local folder.
#### 2. `Dict_2_DF(dictionary_in)`
    Converts a dictionary in the style of a DF into a Pandas Dataframe
    E.g. each column has multiple rows containing the information for an index
    **still a work in progress**

In [ ]:
## Extract daily exports of entire TMBD database
## Function accepts ISO 8601 spec for dates
## e.g. Full_Mov_DB_2_CSV('YYYY-MM-DD')
def Full_Mov_DB_2_CSV(date):
    ## format the fetch url
    db_online_path='http://files.tmdb.org/p/exports/movie_ids_'+date[5:7]+'_'+date[8:10]+'_'+date[:4]+'.json.gz'
    print("Export link:", db_online_path)
    
    ## will throw exception if url does not exist
    try:
        data_raw = pd.read_json(db_online_path, compression='gzip', lines=True)
        print("Writing to file...")
    except:
        print('Error in extracting file from link...\nFile may not yet exist on TMDB.')
        return
    
    ## format the local file name
    db_path='FullDB_'+date+'.csv'
    data_raw.to_csv(db_path)
    print("Exported to local folder:",db_path)

resultlist = movie.search("Star wars")
for i in range(len(resultlist)):
    temp = pd.DataFrame.from_dict(resultlist[i], orient='index')
    df_return = 

def dataframe_from_search(query):
    if type(query) != str:
        print("Enter string as search query")
        return
    else:
        querylist = movie.search(query)
        temp = pd.DataFrame.from_dict(querylist[i])
        for i in range(len(querylist)):
            

print(resultlist[0])
test_df = pd.DataFrame.from_dict(resultlist[0], orient='index')
print(test_df)

# Cleaning up the dataset 
#### Remove entries that are:
- movies release before 2000 (21st century movies in dataset)
- movies with no financial information (0 box office and 0 budget)
- 


In [ ]:
## Create local CSV file
## hashed out this one as local fil for 2021-03-15 has already been created
#### Full_Mov_DB_2_CSV('2021-03-15')

In [ ]:
data_raw = pd.read_csv('FullDB_2021-03-15.csv')
display(data_raw.describe())
display(data_raw.head())

In [ ]:
## Taking a look at what col headers are present in movie details
stats = movie.details(10000)
print(pd.DataFrame(stats),'\n')

In [ ]:
## taking a peek at a movie's stats
## does NOT use IMDB id, uses 'id' <- not sure about where this id comes from
## taking a random movie id
stats = movie.details(805900)
movie_attributes = ['title','original_title','id','imdb_id','release_date','genres','budget','revenue','overview','popularity','spoken_languages','vote_average','production_companies','production_countries']
for stuff in movie_attributes:
    print(stuff,':',stats[stuff])
## the 'casts' key contains too many items, using the next code box to look at casts
## production countries may also have many elements for large movies

In [ ]:
## looking at actors in a specific movie
cast_attributes = ['original_name','gender','popularity','adult']
for thing in stats['casts']['cast']:
    for item in cast_attributes:
        print(item,':',thing[item])
    print()

In [ ]:
##looking at crew members in a specific movie
crew_attributes = ['original_name','id','gender','adult', 'job','department','known_for_department']
cast_count = len(stats['casts']['crew'])
print('Number of cast members:', str(cast_count),'\n')
for item in stats['casts']['crew']:
    for subitem in crew_attributes:
        print(subitem,':',item[subitem])
    print()

In [ ]:
test_df = pd.DataFrame.from_dict(stats['casts']['crew'][0], orient='index')
#print(test_df,'\n')
#test_df.info()

## Test code, still unfinished
## Dictionary to DF process:
## Extract dictionary, place into DataFrame
## Concatenate consec data entries together
## Recast columns into the appropriate data types
new_df = None
for item in stats['casts']['crew']:
    sub_df = pd.DataFrame.from_dict(item, orient='index')
    if new_df is None:
        new_df = sub_df.T
    else:
        new_df = pd.concat([new_df, sub_df.T], sort=True)
new_df = new_df.convert_dtypes()
new_df.info()
new_df

In [ ]:
## shortening the dataframe size so that code does not take forever to run
## 10K entries is apparently too long
## limit to 100 entries for now
data_short = data_raw.head(10)
data_short

In [ ]:
## checking the number of movies with financial data
## only counts movies that have both budget and box office numbers (not zero)
## Looks like the call rate is heavily limited after approximately 5000 calls
## API calls after 6000 are reduced to around 4 per second
## Call API in batches of 5000 or less to get past this bottleneck
## Need to check the off time in between batches

hit=0
total_count=0
total_uncount=0
top=len(data_short['id'])

for item_id in data_short['id']:
    ## visual indication of progress
    time.sleep(0.001)
    if total_count+total_uncount == top-1:
        print('API call complete:   '+str(total_count+total_uncount+1)+'/'+str(top))
    else:
        print('API calls: '+str(total_count+total_uncount+1)+'/'+str(top),end = "\r")
    
    ## skips call if api barfs 
    try:
        mov = movie.details(item_id)
        if(mov['budget'] and mov['revenue']):
            hit += 1
        total_count += 1
    except:
        total_uncount += 1
        
print("Successful calls:   ",str(total_count))
print("Unsuccessful calls: ",str(total_uncount))
print("Percentage of movies with financial statistics: ",str(hit/total_count*100)+'%')

In [ ]:
## Wrapping the above code cell into a function
## pass the 'id' column from daily export CSV into function
## take the first 10 values and increment by an order of magnitude for each iteration

def financialstat_counter(data):
    hit=0
    total_count=0
    total_uncount=0
    top=len(data)

    for item_id in data:
        ## visual indication of progress
        time.sleep(0.001)
        if total_count+total_uncount == top-1:
            print('API call complete:   '+str(total_count+total_uncount+1)+'/'+str(top))
        else:
            print('API calls: '+str(total_count+total_uncount+1)+'/'+str(top),end = "\r")

        ## skips call if api barfs 
        try:
            mov = movie.details(item_id)
            if(mov['budget'] and mov['revenue']):
                hit += 1
            total_count += 1
        except:
            total_uncount += 1
    
    value=hit/total_count*100
    print("Successful calls:   ",str(total_count))
    print("Unsuccessful calls: ",str(total_uncount))
    print("Percentage of movies with financial statistics: ",str(value)+'%\n')
    return value

In [ ]:
rep_list = [1,10]
count=0
x=[]
for rep in rep_list:
    x.append(0.01 * financialstat_counter(random.sample(list(data_raw['id']),rep) ) )
    count+=1
    if count == len(rep_list):
        break
print(x)

In [ ]:
list(data_raw['id'])
data_raw['id'].head(20)